This notebook takes the raw data found in multiple files within the "raw" folder and places them all into one file for a given month. The data is separated into motor vehicle theft and theft and finally geocoded so that it can be used in the map construction.

In [12]:
import pandas as pd
import os
import geocoder
import requests

In [9]:
for folder in os.listdir('./raw'):
    file_list = []
    for file in os.listdir('./raw/' + folder):
        if(file.endswith('.xls')):
            data = pd.read_excel('./raw/' + folder + '/' + file)
            file_list.append(data)
   
    df = pd.concat(file_list, ignore_index=True)
    
    columns = ['Incident Number','Date', 'Time', 'Police District','Offense 1',
          'Offense 2', 'Offense 3', 'Offense 4', 'Offense 5', 'Location']
    df.columns = columns
    
    df = df[df['Offense 1'].notnull()]
    df = df[df['Location'].notnull()]
    
    df.to_csv('./raw/' + folder + 'Complete.csv', sep=';', index = False)

In [13]:
def address_to_latlong(address):
    '''Returns the latitude and longitude for a given address'''
    address += ", Milwaukee, WI"
    location = geocoder.arcgis(address, session = session)
    
    if not location.latlng:
        print("Couldn't Parse: "  + address )
    else:
        print(f'Parsed: {address} - {location.latlng}')
    return location.latlng

In [14]:
for file in os.listdir('./raw/'):
    if(file.endswith('.csv')):
        df = pd.read_csv('./raw/'+ file, sep =';')
    
        zipped = zip(df['Offense 1'].items(), df['Offense 2'].items(), 
        df['Offense 3'].items(), df['Offense 4'].items(), df['Offense 5'].items())
        keyword = 'theft'
        
        for off_1, off_2, off_3, off_4, off_5 in zipped:
            if(not keyword in off_1[1].lower() and
              (pd.isnull(off_2[1]) or not keyword in off_2[1].lower()) and
              (pd.isnull(off_3[1]) or not keyword in off_3[1].lower()) and
              (pd.isnull(off_4[1]) or not keyword in off_4[1].lower()) and
              (pd.isnull(off_5[1]) or not keyword in off_5[1].lower())):
                df = df.drop(off_1[0])
 
        df['Address'] = df['Location'].copy()
            
        theft = df.copy()
        m_theft = df.copy()
        
        zipped = zip(df['Offense 1'].items(), df['Offense 2'].items(), 
        df['Offense 3'].items(), df['Offense 4'].items(), df['Offense 5'].items())
        keyword = 'motor vehicle theft'
        
        for off_1, off_2, off_3, off_4, off_5 in zipped:
            if(not keyword in off_1[1].lower() and
                (pd.isnull(off_2[1]) or not keyword in off_2[1].lower()) and
                (pd.isnull(off_3[1]) or not keyword in off_3[1].lower()) and
                (pd.isnull(off_4[1]) or not keyword in off_4[1].lower()) and
                (pd.isnull(off_5[1]) or not keyword in off_5[1].lower())):
                m_theft = m_theft.drop(off_1[0])
            else:
                theft = theft.drop(off_1[0])
                
        
        with requests.Session() as session:
            # This is likely to be very slow
            # depending on the geocoding service used
            theft['Location'] = theft['Location'].apply(address_to_latlong)
            m_theft['Location'] = m_theft['Location'].apply(address_to_latlong)
            
        theft.to_csv('./' + 'theft_' + file.split('Complete.csv')[0] + '.csv', sep=';', index = False)
        m_theft.to_csv('./' + 'm_theft_' + file.split('Complete.csv')[0] + '.csv', sep=';', index = False)
        print(f'Finished {file}')

Parsed: 4000 N TEUTONIA AV, Milwaukee, WI - [43.08978762106314, -87.93883689398197]
Parsed: 5019 N HOPKINS ST, Milwaukee, WI - [43.108744830688515, -87.96193523483733]
Parsed: 4821 N 22ND ST, Milwaukee, WI - [43.10577604743512, -87.93954421224497]
Parsed: 4840 N 41ST ST, Milwaukee, WI - [43.10552683236193, -87.9638333280341]
Parsed: 4039 N 19TH ST, Milwaukee, WI - [43.09049672745109, -87.93440103375819]
Parsed: 5031 N 35TH ST, Milwaukee, WI - [43.10881558522157, -87.95654415290905]
Parsed: 2800 W VILLARD AV, Milwaukee, WI - [43.11188785896577, -87.94779868196629]
Parsed: 5306 N 36TH ST, Milwaukee, WI - [43.11390541326679, -87.95755533135232]
Parsed: 5376 N 38TH ST, Milwaukee, WI - [43.11546224562872, -87.9599338607827]
Parsed: 4801 N HOPKINS ST, Milwaukee, WI - [43.104664362244066, -87.96002292996054]
Parsed: 5237 N 35TH ST, Milwaukee, WI - [43.11258408700891, -87.95645432016074]
Parsed: 4707 N 20TH ST, Milwaukee, WI - [43.10279069608748, -87.93703011368208]
Parsed: 4026 N 13TH ST, Mil

Parsed: 3133 N 55TH ST, Milwaukee, WI - [43.07605216763807, -87.98203108814687]
Parsed: 6200 W CENTER ST, Milwaukee, WI - [43.06814721890352, -87.9897887189035]
Parsed: 2872 N 55TH ST, Milwaukee, WI - [43.071371413266775, -87.98202241185312]
Parsed: 2712 N 53RD ST, Milwaukee, WI - [43.06832113627445, -87.97979242296164]
Parsed: 5225 W VLIET ST, Milwaukee, WI - [43.04916762493575, -87.97953685763736]
Parsed: 3863 N 62ND ST, Milwaukee, WI - [43.08760858673324, -87.98958163200395]
Parsed: 418 S 65TH ST, Milwaukee, WI - [43.027317167638074, -87.99363444460172]
Parsed: 4900 W BURLEIGH ST, Milwaukee, WI - [43.075338987932845, -87.97483895418657]
Parsed: 2805 N AVONDALE BL, Milwaukee, WI - [43.070064337786306, -87.99253622525804]
Parsed: 5800 W APPLETON AV, Milwaukee, WI - [43.068119288866754, -87.98527521908176]
Parsed: 2653 N 54TH ST, Milwaukee, WI - [43.06729558673322, -87.9810290809335]
Parsed: 1348 N 44TH ST, Milwaukee, WI - [43.04857449708581, -87.96909391185314]
Parsed: 6735 W MOLTKE A

Parsed: 1018 S 18TH ST, Milwaukee, WI - [43.020647999999994, -87.93548341185313]
Parsed: 1131 W GRANT ST, Milwaukee, WI - [43.004543810776795, -87.92626897407132]
Parsed: 240 W VIRGINIA ST, Milwaukee, WI - [43.02639239983439, -87.91319660009621]
Parsed: 255 W BRUCE ST, Milwaukee, WI - [43.025191747210535, -87.91339954806679]
Parsed: 700 W BECHER ST, Milwaukee, WI - [43.00654113364878, -87.92010262419453]
Parsed: 1818 S 6TH ST, Milwaukee, WI - [43.01055782653356, -87.9185544551333]
Parsed: 1202 S CESAR E CHAVEZ DR, Milwaukee, WI - [43.01893905285593, -87.93300951871218]
Parsed: 1037 S 5TH ST, Milwaukee, WI - [43.02074850130819, -87.91688452970698]
Parsed: 914 S 17TH ST, Milwaukee, WI - [43.021777977782996, -87.9342624190665]
Parsed: 1630 S 6TH ST, Milwaukee, WI - [43.013534596804185, -87.91842870911293]
Parsed: 1410 W HAYES AV, Milwaukee, WI - [43.00128951154119, -87.93028333236194]
Parsed: 200 S EMMBER LA, Milwaukee, WI - [43.03041972437939, -87.93058386218969]
Parsed: 1659 S 7TH ST, M

Parsed: 3447 S 8TH ST, Milwaukee, WI - [42.98181350291421, -87.92163851543633]
Parsed: 2932 S 7TH ST, Milwaukee, WI - [42.991163413266776, -87.92016443738835]
Parsed: 2915 S DELAWARE AV, Milwaukee, WI - [42.990996595043114, -87.88261639191516]
Parsed: 2529 S 6TH ST, Milwaukee, WI - [42.99849941909517, -87.91890755539828]
Parsed: 2638 S 9TH ST, Milwaukee, WI - [42.996796148946174, -87.92244697828087]
Parsed: 2705 S 12TH ST, Milwaukee, WI - [42.99532283819033, -87.92737856650677]
Parsed: 434 E WILSON ST, Milwaukee, WI - [43.0000464532574, -87.90371941326677]
Parsed: 3270 S 7TH ST, Milwaukee, WI - [42.98493541326678, -87.92030491185312]
Parsed: 3164 S CLEMENT AV, Milwaukee, WI - [42.98672636081136, -87.89425495181507]
Parsed: 1822 E FERNWOOD AV, Milwaukee, WI - [42.985740015840356, -87.88631121233186]
Parsed: 2416 S BURRELL ST, Milwaukee, WI - [43.000595612268455, -87.90800297013695]
Parsed: 2229 N 35TH ST, Milwaukee, WI - [43.05979609683667, -87.95760813142046]
Parsed: 2423 W LISBON AV, 

Parsed: 2041 N 34TH ST, Milwaukee, WI - [43.05724250291419, -87.956380569825]
Parsed: 3500 W NORTH AV, Milwaukee, WI - [43.06065651875455, -87.9577335]
Parsed: 2324 N 18TH ST, Milwaukee, WI - [43.06103899417161, -87.93502237910455]
Parsed: 2773 N 12TH ST, Milwaukee, WI - [43.06921233527612, -87.92680814643067]
Parsed: 2136 N 12TH ST, Milwaukee, WI - [43.05866332944774, -87.92688393349322]
Parsed: 2678 N 36TH ST, Milwaukee, WI - [43.067510026212865, -87.95855515749534]
Parsed: 1933 N 18TH ST, Milwaukee, WI - [43.05554952844943, -87.93519056650678]
Parsed: 2339 N 40TH ST, Milwaukee, WI - [43.061326339971444, -87.96347884786402]
Parsed: 2100 W BARBEE ST, Milwaukee, WI - [43.05403647879262, -87.9390315]
Parsed: 2465 N 33RD ST, Milwaukee, WI - [43.063866754371276, -87.95525758093349]
Parsed: 1200 W MEINECKE AV, Milwaukee, WI - [43.06223510573828, -87.92692729760711]
Parsed: 1866 N 13TH ST, Milwaukee, WI - [43.05541449708582, -87.92831588631792]
Parsed: 2353 N 45TH ST, Milwaukee, WI - [43.06

...

Parsed: 2127 N 45TH ST, Milwaukee, WI - [43.05772744463039, -87.97008860646872]
Parsed: 3331 W NORTH AV, Milwaukee, WI - [43.060547790558545, -87.95555844763963]
Parsed: 2200 N 30TH ST, Milwaukee, WI - [43.05919941827573, -87.95097565211606]
Parsed: 2237 N 42ND ST, Milwaukee, WI - [43.060075994171626, -87.96587809925535]
Parsed: 2545 N 30TH ST, Milwaukee, WI - [43.06519858673323, -87.95114963979422]
Parsed: 2720 N 17TH ST, Milwaukee, WI - [43.06824900000001, -87.93438638631791]
Parsed: 2227 N 40TH ST, Milwaukee, WI - [43.059780754371275, -87.96348810257359]
Parsed: 2028 W WRIGHT ST, Milwaukee, WI - [43.06414951485942, -87.938424]
Parsed: 1642 N 29TH ST, Milwaukee, WI - [43.05240913627446, -87.94992391185313]
Parsed: 3739 W SARNOW ST, Milwaukee, WI - [43.05425232526414, -87.96079345158394]
Parsed: 2319 N 46TH ST, Milwaukee, WI - [43.061095844018695, -87.97134155539827]
Parsed: 2512 N 35TH ST, Milwaukee, WI - [43.064621497085824, -87.9574239190665]
Parsed: 1960 N 28TH ST, Milwaukee, WI -

Parsed: 4469 N 71ST ST, Milwaukee, WI - [43.098777167638076, -88.00068109925536]
Parsed: 3980 N 60TH ST, Milwaukee, WI - [43.08967440254139, -87.98681407490675]
Parsed: 4741 N 68TH ST, Milwaukee, WI - [43.103686661953915, -87.99680908814686]
Parsed: 4131 N 62ND ST, Milwaukee, WI - [43.09255900582838, -87.98948157372014]
Parsed: 8909 W THURSTON AV, Milwaukee, WI - [43.120405494920355, -88.02342136569678]
Parsed: 8121 W THURSTON AV, Milwaukee, WI - [43.12167987480537, -88.01281640689014]
Parsed: 5736 N 92ND ST, Milwaukee, WI - [43.12202307799066, -88.02704588631792]
Parsed: 6215 N 84TH ST #4, Milwaukee, WI - [43.13101452844941, -88.01564816143431]
Parsed: 7305 W HAMPTON AV, Milwaukee, WI - [43.10487954674261, -88.00353041909516]
Parsed: 5020 W FOND DU LAC AV, Milwaukee, WI - [43.09019448289703, -87.97647325010728]
Parsed: 4402 N 65TH ST, Milwaukee, WI - [43.097408832361936, -87.99310238631791]
Parsed: 8713 W THURSTON AV, Milwaukee, WI - [43.12048952842075, -88.02124166472387]
Parsed: 851

Parsed: 2415 W STATE ST, Milwaukee, WI - [43.0431778126264, -87.94327875983515]
Parsed: 250 W KILBOURN AV, Milwaukee, WI - [43.041590404035645, -87.9140917095348]
Parsed: 2714 W MICHIGAN ST, Milwaukee, WI - [43.03758480054734, -87.94801165436105]
Parsed: 609 N 31ST ST, Milwaukee, WI - [43.037820251457106, -87.9528311281088]
Parsed: 3210 W WELLS ST, Milwaukee, WI - [43.04031950432783, -87.95479980391252]
Parsed: 2920 W HIGHLAND BL, Milwaukee, WI - [43.044640457152525, -87.95129391618097]
Parsed: 3001 W WISCONSIN AV, Milwaukee, WI - [43.03864753173899, -87.951942]
Parsed: 235 E JUNEAU AV, Milwaukee, WI - [43.04568310405102, -87.90940362685109]
Parsed: 1525 N 20TH ST, Milwaukee, WI - [43.05032738983845, -87.9378133580522]
Parsed: 2518 W WISCONSIN AV, Milwaukee, WI - [43.03876947157924, -87.945543]
Parsed: 1207 W VLIET ST, Milwaukee, WI - [43.04852552842077, -87.92746251832186]
Parsed: 1002 N ASTOR ST, Milwaukee, WI - [43.04413817770216, -87.89954349298148]
Parsed: 1113 N 25TH ST, Milwauke

Parsed: 2024 N HOLTON ST, Milwaukee, WI - [43.05710747155058, -87.90532240074464]
Parsed: 1518 W RING ST, Milwaukee, WI - [43.07796349711448, -87.93045930391251]
Parsed: 3551 N 12TH ST, Milwaukee, WI - [43.08277558673322, -87.92603959925536]
Parsed: 2878 N 18TH ST, Milwaukee, WI - [43.071273303912534, -87.93552391185312]
Parsed: 1858 N COMMERCE ST, Milwaukee, WI - [43.054832318858864, -87.90610335204012]
Parsed: 2024 N 1ST ST, Milwaukee, WI - [43.056811639245794, -87.91110083774588]
Parsed: 4800 N IROQUOIS AV, Milwaukee, WI - [43.10413456615716, -87.91556437697615]
Parsed: 2148 N HOLTON ST, Milwaukee, WI - [43.05878046572221, -87.90528445181508]
Parsed: 4229 W BEETHOVEN PL, Milwaukee, WI - [43.098016506780674, -87.96609025145709]
Parsed: 3882 N SHERMAN BL #LOWER, Milwaukee, WI - [43.08764332944776, -87.96700541517136]
Parsed: 4475 N HOPKINS ST, Milwaukee, WI - [43.09870651956261, -87.95754481767541]
Parsed: 3005 N 27TH ST, Milwaukee, WI - [43.07358658673323, -87.94730915032581]
Parsed:

Parsed: 1116 S 35TH ST, Milwaukee, WI - [43.01955867509044, -87.95764979984018]
Parsed: 616 S 28TH ST, Milwaukee, WI - [43.025273999999996, -87.94902293738835]
Parsed: 6105 N 60TH ST, Milwaukee, WI - [43.12829661226846, -87.98597116085739]
Parsed: 11513 W BROWN DEER RD, Milwaukee, WI - [43.177076553379074, -88.05462050138497]
Parsed: 4800 W VILLARD AV, Milwaukee, WI - [43.111986504327824, -87.972822]
Parsed: 5828 N 62ND ST, Milwaukee, WI - [43.123715717179294, -87.98826285356932]
Parsed: 7916 W GREEN TREE RD, Milwaukee, WI - [43.14143352986306, -88.009614]
Parsed: 6567 N 73RD ST, Milwaukee, WI - [43.1371731781983, -88.00209974759062]
Parsed: 6715 N 75TH ST, Milwaukee, WI - [43.13994286372554, -88.00478765032581]
Parsed: 5945 N 60TH ST, Milwaukee, WI - [43.12572277990651, -87.9860116608574]
Parsed: 8641 N SERVITE DR, Milwaukee, WI - [43.174230108777614, -88.01098685225176]
Parsed: 5867 N 61ST ST, Milwaukee, WI - [43.12412294986064, -87.98728706256274]
Parsed: 8270 W CASPER ST, Milwaukee